<a href="https://colab.research.google.com/github/zhaoxiaozhou/colab/blob/main/hook_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def print_tensor(name, tensors):
  if isinstance(tensors, torch.Tensor):
    print("=====================================================================================")
    print("#####################################################################################")
    print(name, tensors.shape, tensors.dtype, tensors)
  elif isinstance(tensors, (list, tuple)):
    for tensor in tensors:
      print_tensor(name, tensor)
  elif isinstance(tensors, dict):
    for key, value in tensors.items():
      print_tensor(name + '.' + key, value)
  else:
    print(name, tensors)

In [ ]:
def hook_func(name, module):
  def hook_function(module, input, output):
    print_tensor(name + "input", input)
    print_tensor(name + "output", output)

In [ ]:
def hook_for_model(model):
  for name, module in model.named_modules():
    module.register_forward_hook(hook_func('[forward]: '+ name, module))
    print("****************************************")
    print(name)
    module.register_backward_hook(hook_func('[backward]: '+ name, module))
    print("****************************************")

In [ ]:
class TestModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Sequential(
      # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
      nn.Linear(50, 16),
      nn.ReLU()
    )
    self.layer2 = nn.Sequential(
      nn.Linear(16, 10)
    )
  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    return x

In [ ]:
model = TestModel()
hook_for_model(model)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for step in range(100):
  optimizer.zero_grad()
  inputs = torch.randn((10, 50))
  outputs = model(inputs)
  labels = torch.zeros((10))
  loss = loss_fn(outputs, labels)
  # print(f"step:{step}, loss:{loss}")
  loss.backward()
  optimizer.step()

****************************************

****************************************
****************************************
layer1
****************************************
****************************************
layer1.0
****************************************
****************************************
layer1.1
****************************************
****************************************
layer2
****************************************
****************************************
layer2.0
****************************************


TypeError: 'NoneType' object is not callable